In [8]:
import requests
URL = 'https://www.encodeproject.org/search/?searchTerm=Chip-Seq&type=experiment&assembly=hg19&target.investigated_as=transcription%20factor&limit=all&format=json'
    

In [9]:
req = requests.get(URL)
result = req.json()
res = result['@graph']

In [23]:
text=''
for x in res:
    expt = x['@id'].rstrip('/').replace('/experiments/','')   
    text+='{}\n'.format(expt)
#text+= res[-1]['@id'].rstrip('/').replace('/experiments/','')   
with open('encode_input.tsv','w') as f:
    f.write(text)

In [19]:
len(res[:-1])

879

In [22]:
res[-1]

{u'@id': u'/experiments/ENCSR000DNO/',
 u'@type': [u'Experiment', u'Dataset', u'Item'],
 u'accession': u'ENCSR000DNO',
 u'assay_term_name': u'ChIP-seq',
 u'award': {u'project': u'ENCODE'},
 u'biosample_term_name': u'GM12878',
 u'description': u'KAT2A ChIP-seq on human GM12878',
 u'files': [],
 u'lab': {u'title': u'Kevin Struhl, HMS'},
 u'replicates': [{u'antibody': {u'accession': u'ENCAB000AHA'},
   u'library': {u'biosample': {u'age': u'unknown',
     u'life_stage': u'adult',
     u'organism': {u'scientific_name': u'Homo sapiens'},
     u'treatments': []}}},
  {u'antibody': {u'accession': u'ENCAB000AHA'},
   u'library': {u'biosample': {u'age': u'unknown',
     u'life_stage': u'adult',
     u'organism': {u'scientific_name': u'Homo sapiens'},
     u'treatments': []}}}],
 u'status': u'released',
 u'target': {u'label': u'KAT2A'}}